In [ ]:
"""
Created on Wed Nov 15 14:58 2023

Exploring data and trying to visualize it

Author: @claraburgard
"""

In [14]:
import xarray as xr
import numpy as np

import cmocean
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns

In [4]:
sns.set_context('paper')

In [22]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [6]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'

In [74]:
T_grid_vars_closed = xr.open_dataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open = xr.open_dataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open = T_grid_vars_open.where(T_grid_vars_open['so'] > 0)

In [75]:
lon = T_grid_vars_closed.nav_lon
lat = T_grid_vars_closed.nav_lat


In [76]:
mask_Ant = lat < -50
T_grid_vars_closed_Ant = T_grid_vars_closed.where(mask_Ant, drop=True)
T_grid_vars_open_Ant = T_grid_vars_open.where(mask_Ant, drop=True)
lon_Ant = T_grid_vars_closed_Ant.nav_lon
lat_Ant = T_grid_vars_closed_Ant.nav_lat

In [77]:
def map_spstereo_compare3(lon,lat,ref,modif,name,legend='yes',lat_lim=-50):

    f = plt.figure(figsize=(8.25,8.25/3))
    #f = plt.figure()
    #f.suptitle(str(time_in.values)[0:16],fontsize=22)
    
    wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)
    wrap_modif, wrap_lon = modif, lon #add_cyclic_point(modif.values,coord=lon,axis=1)

    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpl.path.Path(verts * radius + center)        
            
    ax1 = plt.subplot(1, 3, 1, projection=ccrs.SouthPolarStereo(central_longitude=0))
    abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),cmap=cmocean.cm.thermal,rasterized=True)
    ax1.coastlines(resolution='110m', linewidth=0.5)
    ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
    ax1.set_boundary(circle, transform=ax1.transAxes)


    if legend=='yes':
        cbar = f.colorbar(abso0, ax=ax1, shrink=0.7,orientation='vertical',extend='both')
        cbar.set_label(name,rotation=90)  

    ax2 = plt.subplot(1, 3, 2, projection=ccrs.SouthPolarStereo(central_longitude=0))
    ax2.coastlines(resolution='110m', linewidth=0.5)
    abso = ax2.pcolormesh(wrap_lon,lat,wrap_modif,transform=ccrs.PlateCarree(),cmap=cmocean.cm.thermal,rasterized=True)
    ax2.set_extent([-180, 180, -90, lat_lim], crs=ccrs.PlateCarree())
    #ax2.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    ax2.set_boundary(circle, transform=ax2.transAxes)

    if legend=='yes':
        cbar = f.colorbar(abso0, ax=ax2, shrink=0.7,orientation='vertical',extend='both')
        cbar.set_label(name,rotation=90)    

    ax3 = plt.subplot(1, 3, 3, projection=ccrs.SouthPolarStereo(central_longitude=0))
    ax3.coastlines(resolution='110m', linewidth=0.5)
    diff1 = ax3.pcolormesh(wrap_lon,lat,wrap_modif - wrap_ref,transform=ccrs.PlateCarree(),cmap=mpl.cm.coolwarm,rasterized=True)
    ax3.set_extent([-180, 180, -90, lat_lim], crs=ccrs.PlateCarree())
    #ax3.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
    ax3.set_boundary(circle, transform=ax3.transAxes)

    if legend=='yes':
        cbar = f.colorbar(diff1, ax=ax3, shrink=0.7,orientation='vertical',extend='both')
        cbar.set_label('Difference modif - ref '+name,rotation=90)    


    
    f.tight_layout()
    
    return f


In [78]:
f = map_spstereo_compare3(lon_Ant,lat_Ant,T_grid_vars_closed_Ant['thetao'].isel(deptht=0),T_grid_vars_open_Ant['thetao'].isel(deptht=0),
                          'Temperature [°C]',legend='yes',lat_lim=-50)

In [56]:
T_grid_vars_open_Ant['so'].isel(deptht=50).min()

<xarray.DataArray 'so' ()>
array(0.)
Coordinates:
    deptht   float32 1.516e+03

In [ ]:
fig = plt.figure(figsize=[10, 5])

ax1 = plt.subplot(1, 2, 1, projection=ccrs.SouthPolarStereo())

# Limit the map to -60 degrees latitude and below.
ax1.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())

#ax1.add_feature(cartopy.feature.LAND)
#ax1.add_feature(cartopy.feature.OCEAN)

#ax1.gridlines()

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1.set_boundary(circle, transform=ax1.transAxes)

plt.show()


In [45]:
f = plt.figure(figsize=(8.25,8.25/3))

ax1 = plt.subplot(1, 3, 1, projection=ccrs.SouthPolarStereo(central_longitude=0))
abso0 = ax1.pcolormesh(lon_Ant,lat_Ant,T_grid_vars_closed_Ant['thetao'].isel(deptht=0),transform=ccrs.PlateCarree(),cmap=cmocean.cm.thermal,rasterized=True)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)



In [ ]:
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, lat_lim], crs=ccrs.PlateCarree())
#ax1.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')
ax1.set_boundary(circle, transform=ax1.transAxes)